In [1]:
pip install polars

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import polars as pl
import time

## 1. Prueba de Carga de Datos (Tiempo de Ejecución)

In [3]:
# Medir el tiempo de carga con pandas
start_time = time.time()
df_pandas = pd.read_csv("/kaggle/input/100-million-data-csv/custom_1988_2020.csv")
pandas_time = time.time() - start_time
print(f"Tiempo de carga con pandas: {pandas_time:.4f} segundos")

Tiempo de carga con pandas: 141.5910 segundos


In [4]:
# Medir el tiempo de carga con polars
start_time = time.time()
df_polars = pl.read_csv("/kaggle/input/100-million-data-csv/custom_1988_2020.csv")
polars_time = time.time() - start_time
print(f"Tiempo de carga con polars: {polars_time:.4f} segundos")

Tiempo de carga con polars: 20.7594 segundos


## 2. Prueba de Uso de Memoria (Memory Profiler)

In [5]:
from memory_profiler import memory_usage

def load_pandas():
    return pd.read_csv("/kaggle/input/100-million-data-csv/custom_1988_2020.csv")

def load_polars():
    return pl.read_csv("/kaggle/input/100-million-data-csv/custom_1988_2020.csv")

# Medir uso de memoria con pandas
pandas_mem = memory_usage(load_pandas)
print(f"Uso de memoria con pandas: {max(pandas_mem) - min(pandas_mem):.4f} MB")

# Medir uso de memoria con polars
polars_mem = memory_usage(load_polars)
print(f"Uso de memoria con polars: {max(polars_mem) - min(polars_mem):.4f} MB")

Uso de memoria con pandas: 13805.8203 MB
Uso de memoria con polars: 11661.8516 MB


## 3. Monitoreo de CPU y Memoria con psutil

In [8]:
import psutil

def monitor_performance(func):
    process = psutil.Process()
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss / (1024 * 1024)  # Convertir a MB
    start_time = time.time()

    # Ejecutar la función
    func()

    elapsed_time = time.time() - start_time
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss / (1024 * 1024)  # Convertir a MB

    print(f"Tiempo de ejecución: {elapsed_time:.4f} segundos")
    print(f"CPU usada: {end_cpu - start_cpu:.2f}%")
    print(f"Memoria usada: {end_mem - start_mem:.4f} MB")

# Monitoreo con pandas
monitor_performance(lambda: pd.read_csv("/kaggle/input/100-million-data-csv/custom_1988_2020.csv"))

# Monitoreo con polars
monitor_performance(lambda: pl.read_csv("/kaggle/input/100-million-data-csv/custom_1988_2020.csv"))

Tiempo de ejecución: 114.1603 segundos
CPU usada: 99.90%
Memoria usada: -1.2461 MB
Tiempo de ejecución: 32.6770 segundos
CPU usada: 177.10%
Memoria usada: 1.2969 MB


### 4. PySpark vs Polars

In [6]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=7179957227795ba0961b30746d6f183208a6bc1649efe36ff439289987946596
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [12]:
# Importar librerías necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import polars as pl
import time

# --- Configuración de PySpark ---

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("CSV-Data-Processing") \
    .getOrCreate()

# Leer datos desde el archivo CSV
start_time_read_spark = time.time()
df_spark = spark.read.csv("/kaggle/input/100-million-data-csv/custom_1988_2020.csv", header=True, inferSchema=True)
end_time_read_spark = time.time()

# Mostrar el tiempo de lectura en PySpark
print(f"Tiempo de lectura PySpark: {end_time_read_spark - start_time_read_spark} segundos")

# Asignar nombres de columnas descriptivos
df_spark = df_spark.toDF("col_1", "col_103", "col_100", "col_000000190", "col_0", "col_35843", "col_34353", "col_extra")

# Filtrar los datos en PySpark
start_time_filter_spark = time.time()
filtered_spark = df_spark.filter(col('col_1') > 1000)
result_spark = filtered_spark.groupBy('col_103').count()
end_time_filter_spark = time.time()

# Mostrar resultados en PySpark
print("Resultados PySpark:")
result_spark.show()
print(f"Tiempo de filtrado y agrupación en PySpark: {end_time_filter_spark - start_time_filter_spark} segundos")



Tiempo de lectura PySpark: 148.87319493293762 segundos
Resultados PySpark:


+-------+--------+
|col_103|   count|
+-------+--------+
|      1|69088367|
|      2|44518954|
+-------+--------+

Tiempo de filtrado y agrupación en PySpark: 0.024688243865966797 segundos


In [15]:
import polars as pl
import time

# --- Configuración de Polars ---

# Leer datos desde el archivo CSV en Polars
start_time_read_polars = time.time()
df_polars = pl.read_csv("/kaggle/input/100-million-data-csv/custom_1988_2020.csv")
end_time_read_polars = time.time()

# Mostrar el tiempo de lectura en Polars
print(f"Tiempo de lectura Polars: {end_time_read_polars - start_time_read_polars} segundos")

# Asignar nombres de columnas descriptivos en Polars
df_polars = df_polars.rename({
    "1": "col_1",
    "103": "col_103",
    "100": "col_100",
    "000000190": "col_000000190",
    "0": "col_0",
    "35843": "col_35843",
    "34353": "col_34353"
})

# Filtrar los datos en Polars
start_time_filter_polars = time.time()
filtered_polars = df_polars.filter(pl.col('col_1') > 1000)

# Agrupar y contar en Polars (corrigiendo el método)
result_polars = filtered_polars.groupby('col_103').agg(pl.count().alias('count'))
end_time_filter_polars = time.time()

# Mostrar resultados en Polars
print("Resultados Polars:")
print(result_polars)
print(f"Tiempo de filtrado y agrupación en Polars: {end_time_filter_polars - start_time_filter_polars} segundos")


Tiempo de lectura Polars: 17.000966787338257 segundos


AttributeError: 'DataFrame' object has no attribute 'groupby'

In [ ]:
# Celda 1: PySpark para Extraer Datos de BigQuery

from pyspark.sql import SparkSession
import time

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("BigQuery to DataFrame") \
    .config("spark.sql.extensions", "com.google.cloud.spark.bigquery.BigQuerySparkSessionExtensions") \
    .config("spark.sql.catalog.spark_bigquery", "com.google.cloud.spark.bigquery.BigQueryCatalog") \
    .getOrCreate()

# Medir el tiempo de lectura desde BigQuery
start_time_read_spark = time.time()

# Leer datos desde BigQuery
project_id = "tu_proyecto_id"
dataset_id = "tu_dataset_id"
table_id = "tu_tabla_id"
table_name = f"{project_id}.{dataset_id}.{table_id}"

df_spark = spark.read \
    .format("bigquery") \
    .option("table", table_name) \
    .load()

end_time_read_spark = time.time()

# Mostrar el tiempo de lectura en PySpark
print(f"Tiempo de lectura PySpark: {end_time_read_spark - start_time_read_spark} segundos")

# Mostrar el esquema y los datos
df_spark.printSchema()
df_spark.show()


In [ ]:
# Celda 2: Polars para Extraer Datos de BigQuery

import pandas as pd
import polars as pl
import time

# Medir el tiempo de lectura desde BigQuery
start_time_read_polars = time.time()

# Leer datos desde BigQuery
project_id = "tu_proyecto_id"
query = "SELECT * FROM tu_dataset_id.tu_tabla_id"

df_pandas = pd.read_gbq(query, project_id=project_id)

end_time_read_polars = time.time()

# Convertir a DataFrame de Polars
df_polars = pl.from_pandas(df_pandas)

# Mostrar el tiempo de lectura en Polars
print(f"Tiempo de lectura Polars: {end_time_read_polars - start_time_read_polars} segundos")

# Mostrar el esquema y los datos
print(df_polars)


In [ ]:
!pip install pyspark pandas-gbq polars google-cloud-bigquery
